# Cube Detection Demo (Scenes + OpenCV)

This notebook loads a scenario YAML from `generated_scenarios/`, renders a camera frame via Drake's `MakeHardwareStation`, and runs a simple OpenCV color-based cube detector (with depth sampling for range).


In [ ]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt

from pydrake.all import ImageRgba8U
from manipulation.station import LoadScenario, MakeHardwareStation

# Paths
ROOT = Path(__file__).resolve().parents[2] if '__file__' in globals() else Path.cwd().resolve()
SCENARIO_DIR = ROOT / "src" / "experiments_finalproject" / "generated_scenarios"
print(f"Project root: {ROOT}")
print(f"Scenario dir: {SCENARIO_DIR}")


In [ ]:
# Choose a scenario YAML
available = sorted(p.name for p in SCENARIO_DIR.glob("*.yaml"))
if not available:
    raise FileNotFoundError(f"No YAML files in {SCENARIO_DIR}")
scene_name = available[0]  # pick first by default; change to try others
scene_path = SCENARIO_DIR / scene_name
print(f"Using scene: {scene_name}")


In [ ]:
# Build station from scenario and render a frame
scenario = LoadScenario(filename=str(scene_path))
station = MakeHardwareStation(scenario, meshcat=None)
context = station.CreateDefaultContext()

# Trigger a render so camera images are available
station.ForcedPublish(context)

# List available output ports to find cameras
port_names = [station.get_output_port(i).get_name() for i in range(station.num_output_ports())]
print("Outputs:")
for name in port_names:
    print(" -", name)

# Pick first rgb/depth ports that look like camera outputs
rgb_port_name = next(n for n in port_names if "rgb" in n)
depth_port_name = next(n for n in port_names if "depth" in n)
print(f"Using ports -> rgb: {rgb_port_name}, depth: {depth_port_name}")


In [ ]:
def drake_image_to_rgb(img: ImageRgba8U) -> np.ndarray:
    """Convert Drake ImageRgba8U to HxWx3 uint8 RGB array."""
    if img.size() == 0:
        raise ValueError("Empty image")
    arr = np.array(img.data, copy=False).reshape(img.height(), img.width(), 4)
    return arr[:, :, :3]

# Extract images
rgb_port = station.GetOutputPort(rgb_port_name)
depth_port = station.GetOutputPort(depth_port_name)

rgb_img = rgb_port.Eval(context)
depth_img = depth_port.Eval(context)

rgb = drake_image_to_rgb(rgb_img)
# Depth is ImageDepth32F; reshape to HxW in meters
if depth_img.size() > 0:
    depth = np.array(depth_img.data).reshape(depth_img.height(), depth_img.width())
else:
    depth = None

print(f"RGB shape: {rgb.shape}, depth: {None if depth is None else depth.shape}")

plt.figure(figsize=(6,5))
plt.imshow(rgb)
plt.title(f"Scene: {scene_name}")
plt.axis('off');


In [ ]:
# Simple cube detection via color thresholding (tune for your cube color)
# Example: orange foam brick
hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
# Adjust these bounds to your cube color
lower = np.array([5, 80, 80])   # hue,sat,val lower
upper = np.array([25, 255, 255])
mask = cv2.inRange(hsv, lower, upper)

# Find largest contour
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if not contours:
    raise RuntimeError("No cube-like contour found; adjust HSV bounds")
contours = sorted(contours, key=cv2.contourArea, reverse=True)
main = contours[0]
x, y, w, h = cv2.boundingRect(main)
cx = int(x + w/2)
cy = int(y + h/2)

# Depth at centroid (if depth available)
centroid_depth = float(depth[cy, cx]) if depth is not None else None

# Draw result
vis = rgb.copy()
cv2.rectangle(vis, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.circle(vis, (cx, cy), 4, (255, 0, 0), -1)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1); plt.imshow(rgb); plt.title('RGB'); plt.axis('off')
plt.subplot(1,2,2); plt.imshow(vis); plt.title(f'Detected cube (depth={centroid_depth:.3f} m)' if centroid_depth else 'Detected cube'); plt.axis('off')
plt.tight_layout();

print(f"Bounding box: x={x}, y={y}, w={w}, h={h}; depth={centroid_depth}")


## Notes
- Adjust HSV bounds (`lower`, `upper`) to your cube color if detection fails.
- Depth sampling uses the centroid; for robustness, consider median depth over the mask region.
- If port names differ, tweak the heuristics in the port selection cell (look for names containing `rgb` / `depth`).
- To try another scene, set `scene_name` in the selection cell.
